In [2]:
#import neccessary libraries
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')  # Download tokenizer data
nltk.download('punkt')  # Download tokenizer data (if you haven't already)
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Read the Excel file
file_path = "/content/Input _new.xlsx"  # Replace with your file path
# Load the Excel file into a DataFrame
df = pd.read_excel(file_path)

In [4]:
df

,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...


In [5]:

# Function to extract the title and content from a webpage
def extract_article_title_and_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the title from <h1> with class "entry-title"
        title_tag = soup.find('h1', {'class': 'entry-title'})
        title = title_tag.get_text(strip=True) if title_tag else "No Title Found"

        # Extract the article text from <div> with class "td-post-content tagdiv-type"
        article_content = ''
        div_tag = soup.find('div', {'class': 'td-post-content tagdiv-type'})
        if div_tag:
            article_content = div_tag.get_text(separator="\n", strip=True)

        return title, article_content
    except Exception as e:
        return None, f"Error fetching article: {e}"


# Output folder to save extracted articles
output_folder = "extracted_articles"
os.makedirs(output_folder, exist_ok=True)

# Iterate through the URLs in the DataFrame
for _, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Extract title and content
    title, content = extract_article_title_and_content(url)

    # Combine title and content into one string
    combined_content = f"{title}\n\n{content}" if content else "Content not found."

    # Save the combined content to a file
    file_path = os.path.join(output_folder, f"{url_id}.txt")
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(combined_content)

# Path to the folder containing .txt files
input_folder = output_folder
output_file = "output.xlsx"

# List to store the structured data
data = []

# Iterate over all .txt files in the folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(input_folder, file_name)
        file_id = os.path.splitext(file_name)[0]  # File name without extension

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                combined_content = file.read().strip()

                # Append the extracted data to the list
                data.append({"id": file_id, "combined_content": combined_content})

        except Exception as e:
            print(f"Error reading file {file_name}: {e}")

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Save the structured data to an Excel file
df.to_excel(output_file, index=False)

print(f"Structured data saved to {output_file}")


Structured data saved to structured_data_combined.xlsx


In [5]:
df

,id,combined_content
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...
2,Netclan20241018,Enhancing Front-End Features and Functionality...
3,Netclan20241020,Efficient Processing and Analysis of Financial...


In [6]:
df['combined_content'][1]

'ROAS Dashboard for Campaign-Wise Google Ads Budget Tracking Using Google Ads AP\n\nClient Background\nClient:\nA leading IT & tech firm in the USA\nIndustry Type:\nIT\nProducts & Services:\nIT Consulting, IT Support, SaaS\nOrganization Size:\n200+\nThe Problem\nCreate a ROASing dashboard to fetch google ads budget spent data using google ads api(campaign-wise). The challenge is to develop a ROAS (Return on Ad Spend) dashboard that efficiently retrieves and displays Google Ads budget spent data on a campaign-wise basis using the Google Ads API. The current system lacks a streamlined method for tracking and analyzing ad spend across various campaigns, leading to difficulties in assessing performance and optimizing budget allocation. The goal is to create a comprehensive dashboard that accurately fetches and visualizes budget data, enabling more effective analysis and decision-making. This involves integrating with the Google Ads API, ensuring data accuracy, and providing clear, actionab

In [7]:
df

,id,combined_content
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...
2,Netclan20241018,Enhancing Front-End Features and Functionality...
3,Netclan20241020,Efficient Processing and Analysis of Financial...


In [8]:
df = df.rename(columns={'combined_content': 'content'})

In [9]:
df

,id,content
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...
2,Netclan20241018,Enhancing Front-End Features and Functionality...
3,Netclan20241020,Efficient Processing and Analysis of Financial...


In [9]:
df['content'][0]

'AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy\n\nClient Background\nClient:\nA leading IT & tech firm in the USA\nIndustry Type:\nIT\nProducts & Services:\nIT Consulting, IT Support, SaaS, Marketing Strategy\nOrganization Size:\n10+\nThe Problem\nBuilding AI and ML based YouTube analytics and content creation tool that will help youtuber to understand their subscriber’s watching behaviour, help them in content research, creation and publication.\nOur Solution\nCreated a MERN stack web application and integrated AI models to helps youtuber to generated titles, descriptions, tags, hashtags, captions etc. Help them to check thumbnail quality, analysis on the videos using video auditor tool, analysis on comments using sentiments analysis, help to under their subscribers using churn predication AI model.\nSolution Architecture\nhttps://www.figma.com/file/WQs01mmmNBZ1SjNE2IV8Sl/Youtube-Web-App-By-SHiV?type=design&node-i

In [10]:
def remove_common_section(text):
    # Remove "Project Snapshots" and everything following it until the next section (e.g., "Summarize" or "Contact Details")
    text = re.sub(r'Project Snapshots.*?(?=Summarize|Contact Details)', '', text, flags=re.DOTALL)

    # Remove the "Contact Details" and everything following it until the next section
    text = re.sub(r'Contact Details.*', '', text, flags=re.DOTALL)
    text = re.sub(r'Project website url\s*https?://\S+', '', text)

    # Remove Summarize section (including "This project was done by the Blackcoffer Team")
    text = re.sub(r'Summarize.*?This project was done by the Blackcoffer Team, a Global IT Consulting firm\.', '', text, flags=re.DOTALL)

    # Remove specific sentence about project discussions and daily updates
    text = re.sub(r'For project discussions and daily updates, would you like to use Slack, Skype, Telegram, or Whatsapp\? Please recommend, what would work best for you\.', '', text)

    return text

In [11]:
df['content_cleaned'] = df['content'].apply(remove_common_section)
df

,id,content,content_cleaned
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,AI and ML-Based YouTube Analytics and Content ...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,ROAS Dashboard for Campaign-Wise Google Ads Bu...
2,Netclan20241018,Enhancing Front-End Features and Functionality...,Enhancing Front-End Features and Functionality...
3,Netclan20241020,Efficient Processing and Analysis of Financial...,Efficient Processing and Analysis of Financial...


In [12]:
df['content_cleaned'][0]

'AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy\n\nClient Background\nClient:\nA leading IT & tech firm in the USA\nIndustry Type:\nIT\nProducts & Services:\nIT Consulting, IT Support, SaaS, Marketing Strategy\nOrganization Size:\n10+\nThe Problem\nBuilding AI and ML based YouTube analytics and content creation tool that will help youtuber to understand their subscriber’s watching behaviour, help them in content research, creation and publication.\nOur Solution\nCreated a MERN stack web application and integrated AI models to helps youtuber to generated titles, descriptions, tags, hashtags, captions etc. Help them to check thumbnail quality, analysis on the videos using video auditor tool, analysis on comments using sentiments analysis, help to under their subscribers using churn predication AI model.\nSolution Architecture\nhttps://www.figma.com/file/WQs01mmmNBZ1SjNE2IV8Sl/Youtube-Web-App-By-SHiV?type=design&node-i

In [14]:
!mkdir StopWords

In [13]:
stop_words_folder = "/content/StopWords"

In [14]:
STOP_WORDS_FOLDER_PATH=Path(stop_words_folder)

In [15]:
STOP_WORDS_FOLDER_PATH

PosixPath('/content/StopWords')

In [22]:
def clean_with_stopwords(text, stop_words_folder):
    # Initialize stopwords set
    stop_words = set()

    try:
        # Read stop words from files in the given folder
        for file_path in os.listdir(stop_words_folder):
            if file_path.endswith(".txt"):  # Only process text files
                filepath = os.path.join(stop_words_folder, file_path)
                with open(filepath, 'r', encoding='latin-1') as file:
                    for line in file:
                        # Skip URLs (anything starting with http or https)
                        if re.match(r'http[s]?://', line.strip().lower()):
                            continue

                        # Split stopwords by '|' and strip extra spaces, remove empty strings, and add to stopwords set
                        words = [word.strip().lower() for word in line.split('|') if word.strip()]
                        stop_words.update(words)
    except FileNotFoundError:
        print(f"Error: Stop words folder not found at {stop_words_folder}")
        return text  # Return original text if folder not found
    except Exception as e:
        print(f"An error occurred while reading stop words: {e}")
        return text  # Return original text on error

    # Remove URLs from the input text as well
    text_without_urls = re.sub(r'http[s]?://\S+', '', text)

    # Remove any instances of '|' in the text
    text_without_urls = text_without_urls.replace("|", "")

    # Tokenize the input text into words
    words = word_tokenize(text_without_urls.lower())

    # Remove stop words from the tokenized list
    cleaned_words = [word for word in words if word not in stop_words]

    # Rebuild the cleaned text and remove extra spaces
    cleaned_text = " ".join(cleaned_words)

    # Remove leading/trailing spaces and extra | symbols
    cleaned_text = " ".join(cleaned_text.split())

    return cleaned_text


In [23]:
df

,id,content,content_cleaned
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,AI and ML-Based YouTube Analytics and Content ...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,ROAS Dashboard for Campaign-Wise Google Ads Bu...
2,Netclan20241018,Enhancing Front-End Features and Functionality...,Enhancing Front-End Features and Functionality...
3,Netclan20241020,Efficient Processing and Analysis of Financial...,Efficient Processing and Analysis of Financial...


In [24]:
df['content_cleaned'][1]

'ROAS Dashboard for Campaign-Wise Google Ads Budget Tracking Using Google Ads AP\n\nClient Background\nClient:\nA leading IT & tech firm in the USA\nIndustry Type:\nIT\nProducts & Services:\nIT Consulting, IT Support, SaaS\nOrganization Size:\n200+\nThe Problem\nCreate a ROASing dashboard to fetch google ads budget spent data using google ads api(campaign-wise). The challenge is to develop a ROAS (Return on Ad Spend) dashboard that efficiently retrieves and displays Google Ads budget spent data on a campaign-wise basis using the Google Ads API. The current system lacks a streamlined method for tracking and analyzing ad spend across various campaigns, leading to difficulties in assessing performance and optimizing budget allocation. The goal is to create a comprehensive dashboard that accurately fetches and visualizes budget data, enabling more effective analysis and decision-making. This involves integrating with the Google Ads API, ensuring data accuracy, and providing clear, actionab

In [25]:
text="AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy Client Background Client: A leading IT & tech firm in the USA Industry Type: IT Products & Services: IT Consulting, IT Support, SaaS, Marketing Strategy Organization Size: 10+ The Problem Building AI and ML based YouTube analytics and content creation tool that will help youtuber to understand their subscriber’s watching behaviour, help them in content research, creation and publication. Our Solution Created a MERN stack web application and integrated AI models to helps youtuber to generated titles, descriptions, tags, hashtags, captions etc. Help them to check thumbnail quality, analysis on the videos using video auditor tool, analysis on comments using sentiments analysis, help to under their subscribers using churn predication AI model. Solution Architecture https://www.figma.com/file/WQs01mmmNBZ1SjNE2IV8Sl/Youtube-Web-App-By-SHiV?type=design&node-id=0-1&mode=design&t=Lh2jRx4bGQq6l4WU-0 Deliverables Web Applications Supports Maintenance Feature Enhancement Tech Stack Tools used VS code Language/techniques used React.js Express.js Node.js Python Models used Python libraries Skills used Data scientise Full Stack developer Databases used MongoDB Web Cloud Servers used Google Cloud Platform"
cleaned_text = clean_with_stopwords(text, stop_words_folder)
print(cleaned_text)
len(cleaned_text)

ml-based youtube analytics content creation tool optimizing subscriber engagement content strategy client background client : leading & tech firm industry type : products & services : consulting , support , saas , marketing strategy organization size : 10+ problem building ml based youtube analytics content creation tool youtuber understand subscriber ’ watching behaviour , content research , creation publication . solution created mern web application integrated models helps youtuber generated titles , descriptions , tags , hashtags , captions . check thumbnail quality , analysis videos video auditor tool , analysis comments sentiments analysis , subscribers churn predication model . solution architecture deliverables web applications supports maintenance feature enhancement tech tools code language/techniques react.js express.js node.js python models python libraries skills data scientise full developer databases mongodb web servers google platform


964

In [26]:
df['content_cleaned'] = df['content_cleaned'].apply(lambda x: clean_with_stopwords(x, stop_words_folder))
df

,id,content,content_cleaned
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...


In [27]:
df['content_cleaned'][1]

'roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehou

In [29]:
!mkdir MasterDictionary

In [28]:

def create_positive_negative_dicts(master_dict_folder, stop_words_folder):
    # Helper function to load words from a file
    def load_words_from_file(file_path):
        words = set()
        try:
            with open(file_path, 'r', encoding='latin-1') as file:
                for line in file:
                    word = line.strip().lower()
                    if word:  # Ignore empty lines
                        words.add(word)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred reading file {file_path}: {e}")
        return words

    # Load stop words from the folder
    stop_words = set()
    for file_name in os.listdir(stop_words_folder):
        if file_name.endswith(".txt"):  # Only process .txt files
            file_path = os.path.join(stop_words_folder, file_name)
            stop_words.update(load_words_from_file(file_path))

    # Initialize dictionaries for positive and negative words
    positive_words = set()
    negative_words = set()

    # Load positive words
    positive_file_path = os.path.join(master_dict_folder, "positive-words.txt")
    positive_words.update(load_words_from_file(positive_file_path))

    # Load negative words
    negative_file_path = os.path.join(master_dict_folder, "negative-words.txt")
    negative_words.update(load_words_from_file(negative_file_path))

    # Remove stopwords from positive and negative words
    positive_words = positive_words - stop_words
    negative_words = negative_words - stop_words

    return {
        "positive": sorted(positive_words),
        "negative": sorted(negative_words),
    }




In [29]:
# Example usage
master_dict_folder = "/content/MasterDictionary"
stop_words_folder = "/content/StopWords"
result = create_positive_negative_dicts(master_dict_folder, stop_words_folder)

# Access the dictionaries
positive_dict = result['positive']
negative_dict = result['negative']

# Print results
print(f"Positive Words ({len(positive_dict)}): {positive_dict}")
print(f"Negative Words ({len(negative_dict)}): {negative_dict}")


Positive Words (1907): ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agi

In [31]:
df

,id,content,content_cleaned
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...


In [48]:
#Extracting Derived variables
def calculate_scores(text, positive_dict, negative_dict):
    if not isinstance(text, str):
        return None  # Handle non-string input

    try:
        words = re.findall(r'\b\w+\b', text)

    except Exception as e:
        print(f"Error during tokenization: {e}")
        return None

    positive_score = 0
    negative_score = 0

    for word in words:
        if word in positive_dict:
            positive_score += 1
        elif word in negative_dict:
            negative_score += 1

    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)

    return {
        "positive_score": positive_score,
        "negative_score": negative_score,
        "polarity_score": polarity_score,
        "subjectivity_score": subjectivity_score
    }



In [49]:
# Example Usage
text ="roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = calculate_scores(text,positive_dict,negative_dict)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: {'positive_score': 11, 'negative_score': 4, 'polarity_score': 0.46666663555555765, 'subjectivity_score': 0.056603773371306516}


In [50]:
df[['positive_score', 'negative_score', 'polarity_score', 'subjectivity_score']] = df['content_cleaned'].apply(lambda x: pd.Series(calculate_scores(x, positive_dict, negative_dict))).fillna(0) #fillna added

In [51]:
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348


In [65]:
df['content_cleaned'][1]

'roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehou

In [69]:
def analyze_readability(text):
    """Calculates readability metrics (avg sentence length, % complex words, fog index)."""
    if not isinstance(text, str) or not text.strip():
        return None

    try:
        sentences = sent_tokenize(text)
        words = re.findall(r'\b\w+\b', text)
    except Exception as e:
        print(f"Error during tokenization: {e}")
        return None

    num_sentences = len(sentences)
    num_words = len(words)
    #print(num_sentences)
    #print(num_words)
    if num_sentences == 0 or num_words == 0:
        return {'avg_sentence_length': 0, 'percentage_complex_words': 0, 'fog_index': 0}

    avg_sentence_length = num_words / num_sentences

    def is_complex(word):
        vowels = "aeiouy"
        syllable_count = 0
        for letter in word.lower():
            if letter in vowels:
                syllable_count += 1
        return syllable_count > 2

    complex_words = [word for word in words if is_complex(word)]
    num_complex_words = len(complex_words)
    percentage_complex_words = (num_complex_words / num_words) * 100 if num_words > 0 else 0

    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return {
        'avg_sentence_length': avg_sentence_length,
        'percentage_complex_words': percentage_complex_words,
        'fog_index': fog_index
    }



In [70]:
# Example Usage
text ="roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = analyze_readability(text)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: {'avg_sentence_length': 44.166666666666664, 'percentage_complex_words': 42.64150943396226, 'fog_index': 34.723270440251575}


In [71]:
df[['avg_sentence_length', 'percentage_complex_words', 'fog_index']] = df['content_cleaned'].apply(lambda x: pd.Series(analyze_readability(x))).fillna(0)

In [72]:
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268


In [75]:
def average_words_per_sentence(text):

    if not isinstance(text, str) or not text.strip():
        return 0

    # Split text into sentences using common sentence-ending punctuations
    sentences = sent_tokenize(text)

    # Remove empty strings from the list of sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    if not sentences:  # If no sentences exist
        return 0

    # Count the total number of words
    words = re.findall(r'\b\w+\b', text)
    total_words = len(words)

    # Calculate the average number of words per sentence
    avg_words_per_sentence = total_words / len(sentences)
    return round(avg_words_per_sentence, 2)



In [78]:
# Example Usage
text = "roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = average_words_per_sentence(text)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: 44.17


In [79]:
df['avg_words_per_sentence'] = df['content_cleaned'].apply(average_words_per_sentence)
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037,10.97
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268,15.79


In [98]:
#Complex Word Count
def count_complex_words(text):
    words = re.findall(r'\b\w+\b', text)
    if not words: #Handle no words case
        return 0

    def is_complex(word):
        vowels = "aeiouy"
        syllable_count = 0
        for letter in word:
            if letter in vowels:
                syllable_count += 1
        return syllable_count > 2

    complex_word_count = sum(1 for word in words if is_complex(word))
    return complex_word_count



In [99]:
# Example Usage
text = "roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = count_complex_words(text)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: 113


In [101]:
df['complex_word_count'] = df['content_cleaned'].apply(count_complex_words)
df.head(2)

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50,59
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17,113


In [102]:
df['content_cleaned'][0]

'ml-based youtube analytics content creation tool optimizing subscriber engagement content strategy client background client : leading & tech firm industry type : products & services : consulting , support , saas , marketing strategy organization size : 10+ problem building ml based youtube analytics content creation tool youtuber understand subscriber ’ watching behaviour , content research , creation publication . solution created mern web application integrated models helps youtuber generated titles , descriptions , tags , hashtags , captions . check thumbnail quality , analysis videos video auditor tool , analysis comments sentiments analysis , subscribers churn predication model . solution architecture deliverables web applications supports maintenance feature enhancement tech tools code language/techniques react.js express.js node.js python models python libraries skills data scientise full developer databases mongodb web servers google platform'

In [119]:

def count_cleaned_words(text):

    stop_words = set(stopwords.words('english'))
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize text into words
    words = word_tokenize(text)
    # Remove stopwords and punctuation
    cleaned_words = [
        word for word in words
        if word not in stop_words and word not in string.punctuation
    ]

    # Return the total number of cleaned words
    return len(cleaned_words)


In [120]:
# Example Usage
text = "roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = count_cleaned_words(text)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: 253


In [124]:
# Example Usage
sample_text =  """
client background client : leading automobile & tech firm industry type : automobiles products & services : manufacturing & dealership , financial services organization size : 200+ problem client , toyota dealership management firm , faced significant challenges efficiently processing analyzing financial data extracted pdf files . documents contained crucial information , expenses , financial metrics departments , parsing extracting data accurately proved daunting task . primary issues included inconsistency pdf formatting , difficulty table extraction , ensuring data integrity processing pipeline . solution address challenges , developed comprehensive solution tailored specifically parsing financial data toyota dealership pdf documents . solution comprised series modular components , designed handle specific aspects data processing pipeline . utilized advanced pdf parsing libraries pdfplumber extract tables metadata accurately . additionally , implemented custom algorithms data cleaning validation ensure integrity accuracy extracted data . solution architecture architecture solution designed modularity scalability mind . consisted components : pdf parsing module : responsible extracting tables metadata pdf documents pdfplumber . data cleaning validation module : implemented custom algorithms clean validate extracted data , ensuring consistency accuracy . data aggregation analysis module : utilized pandas aggregating analyzing financial metrics departments time periods . mongodb integration : stored structured financial data mongodb collections efficient storage retrieval . deliverables custom python scripts pdf parsing data processing tailored toyota dealership documents . structured financial data stored mongodb collections , ensuring easy access retrieval . comprehensive documentation detailing system architecture , usage guidelines , maintenance procedures . tech tools pdfplumber , pandas , mongodb language/techniques python , data cleaning , aggregation models custom parsing algorithms skills data processing , python programming databases mongodb web servers gcp technical challenges faced project execution variability pdf document formats : toyota dealership documents exhibited varying formatting , making consistent parsing challenging . handling volumes pdf files : processing number pdf files efficiently compromising performance significant challenge . ensuring data consistency accuracy : maintaining data integrity processing pipeline , presence inconsistent erroneous data , required careful handling . technical challenges solved developed custom parsing algorithms capable handling variability pdf document formats , ensuring consistent accurate extraction financial data . implemented optimized file handling techniques efficiently process volumes pdf files , minimizing processing time resource utilization . employed rigorous data cleaning validation routines identify rectify inconsistencies errors extracted data , ensuring integrity accuracy . business impact streamlined financial data processing toyota dealerships , resulting improved operational efficiency decision-making . enhanced data accuracy reliability facilitated insights dealership performance financial health . reduced effort processing time , enabling stakeholders focus strategic tasks mundane data processing activities . summarize summarized : project blackcoffer team , global consulting firm . contact details solution designed developed blackcoffer team contact details : firm : blackcoffer pvt . ltd. firm website : www.blackcoffer.com firm address : 4/2 , e-extension , shaym vihar phase 1 , 110043 email : ajay @ blackcoffer.com skype : asbidyarthy whatsapp : +91 9717367468 telegram : @ asbidyarthy
"""
avg_words = count_cleaned_words(sample_text)
print(f"Average Words Per Sentence: {avg_words}")

Average Words Per Sentence: 403


In [125]:
df['word_count'] = df['content_cleaned'].apply(count_cleaned_words)
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50,59,109
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17,113,253
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037,10.97,171,358
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268,15.79,247,377


In [169]:
# Syllable counting function
def count_syllables(word):

    word = re.sub(r'[^a-z]', '', word)  # Remove non-alphabetic characters
    if not word:
        return 0
    vowels = "aeiou"
    if word.endswith("es") or word.endswith("ed"):
        word = word[:-2]
    syllable_count = sum(1 for char in word if char in vowels)
    return max(syllable_count, 1)

# Function to calculate total syllable count for a text
def calculate_syllables(text):
    words = re.findall(r'\b\w+\b', text)  # Split text into words
    return sum(count_syllables(word) for word in words)


In [174]:
# Example Usage
text = "roas  dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
avg_words = calculate_syllables(text)
print(f"Average Words Per Sentence: {avg_words}")


Average Words Per Sentence: 597


In [171]:
# Example usage
sample_text =  "client background client : leading automobile & tech firm industry type : automobiles products & services : manufacturing & dealership , financial services organization size : 200+ problem client , toyota dealership management firm , faced significant challenges efficiently processing analyzing financial data extracted pdf files . documents contained crucial information , expenses , financial metrics departments , parsing extracting data accurately proved daunting task . primary issues included inconsistency pdf formatting , difficulty table extraction , ensuring data integrity processing pipeline . solution address challenges , developed comprehensive solution tailored specifically parsing financial data toyota dealership pdf documents . solution comprised series modular components , designed handle specific aspects data processing pipeline . utilized advanced pdf parsing libraries pdfplumber extract tables metadata accurately . additionally , implemented custom algorithms data cleaning validation ensure integrity accuracy extracted data . solution architecture architecture solution designed modularity scalability mind . consisted components : pdf parsing module : responsible extracting tables metadata pdf documents pdfplumber . data cleaning validation module : implemented custom algorithms clean validate extracted data , ensuring consistency accuracy . data aggregation analysis module : utilized pandas aggregating analyzing financial metrics departments time periods . mongodb integration : stored structured financial data mongodb collections efficient storage retrieval . deliverables custom python scripts pdf parsing data processing tailored toyota dealership documents . structured financial data stored mongodb collections , ensuring easy access retrieval . comprehensive documentation detailing system architecture , usage guidelines , maintenance procedures . tech tools pdfplumber , pandas , mongodb language/techniques python , data cleaning , aggregation models custom parsing algorithms skills data processing , python programming databases mongodb web servers gcp technical challenges faced project execution variability pdf document formats : toyota dealership documents exhibited varying formatting , making consistent parsing challenging . handling volumes pdf files : processing number pdf files efficiently compromising performance significant challenge . ensuring data consistency accuracy : maintaining data integrity processing pipeline , presence inconsistent erroneous data , required careful handling . technical challenges solved developed custom parsing algorithms capable handling variability pdf document formats , ensuring consistent accurate extraction financial data . implemented optimized file handling techniques efficiently process volumes pdf files , minimizing processing time resource utilization . employed rigorous data cleaning validation routines identify rectify inconsistencies errors extracted data , ensuring integrity accuracy . business impact streamlined financial data processing toyota dealerships , resulting improved operational efficiency decision-making . enhanced data accuracy reliability facilitated insights dealership performance financial health . reduced effort processing time , enabling stakeholders focus strategic tasks mundane data processing activities ."
cleaned_word_count = calculate_syllables(sample_text)
print(f"syllable word count: {cleaned_word_count}")

syllable word count: 1012


In [173]:
df['syllable_per_word'] = df['content_cleaned'].apply(calculate_syllables)
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50,59,109,292
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17,113,253,597
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037,10.97,171,358,897
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268,15.79,247,377,1061


In [179]:
df['content_cleaned'][1]

'roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehou

In [189]:
#Personal Pronouns
def count_personal_pronouns(text):
    if not isinstance(text, str):
        return None

    pronoun_pattern = r"\b(i|we|my|ours|us)\b"  # \b ensures word boundaries
    matches = re.findall(pronoun_pattern, text.lower())

    #Exclude US
    pronouns = [match for match in matches if match != "us"]

    return len(pronouns)


In [191]:
# Example usage
sample_text = "roas us dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
count_personal = count_personal_pronouns(sample_text)
count_personal

0

In [192]:
# Example usage
sample_text = "roas we i dashboard campaign-wise google ads budget I tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
count_personal = count_personal_pronouns(sample_text)
count_personal

3

In [193]:
df['personal_pronoun_count'] = df['content_cleaned'].apply(count_personal_pronouns)
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronoun_count
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50,59,109,292,0
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17,113,253,597,0
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037,10.97,171,358,897,0
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268,15.79,247,377,1061,0


In [204]:
def calculate_average_word_length(text):
    if not isinstance(text, str) or not text.strip():
        return None
    words = re.findall(r'\b\w+\b', text)
    if not words:
        return 0
    total_characters = sum(len(word) for word in words)
    avg_word= total_characters/ len(words)
    return avg_word

In [211]:
# Example usage
text="roas dashboard campaign-wise google ads budget tracking google ads ap client background client : leading & tech firm industry type : products & services : consulting , support , saas organization size : 200+ problem create roasing dashboard fetch google ads budget spent data google ads api ( campaign-wise ) . challenge develop roas ( return ad spend ) dashboard efficiently retrieves displays google ads budget spent data campaign-wise basis google ads api . current system lacks streamlined method tracking analyzing ad spend campaigns , leading difficulties assessing performance optimizing budget allocation . goal create comprehensive dashboard accurately fetches visualizes budget data , enabling effective analysis decision-making . involves integrating google ads api , ensuring data accuracy , providing clear , actionable insights campaign management . 4o mini solution data source access a. google ads accounts access google platform a. bigquery b. vm setup google big query data warehouse design db schema / models develop python etl tool pull data source save data warehouse deploy etl tool vm run cron update data db frequently a. check frequency run b. recommended 3 6 times c. etl tool run automatically set intervals update db d. add integration module notifications cronjob failure design access dashboard implement dashboard a. vuexy react template b. list kpi display ui c. list charts display ui implement backend api needed build dashboard solution architecture deliverables etl tool dashboard documentations source codes support maintenance tech tools code digital language/techniques python google big query google platform google ads api react.js node.js github models python libraries ( pandas ) skills data scientist full developer databases mysql web servers digital technical challenges faced project execution google ads campaigns data technical challenges solved google ads apis"
average_word_length = calculate_average_word_length(text)
average_word_length

5.981132075471698

In [209]:
df['average_word_length'] = df['content_cleaned'].apply(calculate_average_word_length)
df

,id,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronoun_count,average_word_length
0,Netclan20241017,AI and ML-Based YouTube Analytics and Content ...,ml-based youtube analytics content creation to...,5.0,1.0,0.666667,0.052632,28.500000,51.754386,32.101754,28.50,59,109,292,0,7.070175
1,Netclan20241019,ROAS Dashboard for Campaign-Wise Google Ads Bu...,roas dashboard campaign-wise google ads budget...,11.0,4.0,0.466667,0.056604,44.166667,42.641509,34.723270,44.17,113,253,597,0,5.981132
2,Netclan20241018,Enhancing Front-End Features and Functionality...,enhancing front-end features functionality imp...,12.0,8.0,0.200000,0.053619,10.970588,45.844504,22.726037,10.97,171,358,897,0,6.678284
3,Netclan20241020,Efficient Processing and Analysis of Financial...,efficient processing analysis financial data p...,23.0,12.0,0.314286,0.092348,15.791667,65.171504,32.385268,15.79,247,377,1061,0,7.881266


In [51]:
"""
df['average_word_length'] = df['content_cleaned'].apply(calculate_average_word_length)
df
"""

,id,title,content,content_cleaned,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,percentage_complex_words,fog_index,avg_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_pronoun_count,average_word_length
0,Netclan20241020,Title: Efficient Processing and Analysis of Fi...,Client Background\nClient:\nA leading automobi...,client background client : leading automobile ...,22.0,11.0,0.333333,0.067762,18.037037,52.156057,28.077238,18.037037,254,403,1105,0,7.692683
1,Netclan20241017,Title: AI and ML-Based YouTube Analytics and C...,Client Background\nClient:\nA leading IT & tec...,client background client : leading & tech firm...,5.0,1.0,0.666667,0.030000,33.333333,38.000000,28.533333,33.333333,76,162,395,0,6.711765
2,Netclan20241018,Title: Enhancing Front-End Features and Functi...,Client Background\nClient:\nA leading hospital...,client background client : leading hospital ch...,9.0,8.0,0.058824,0.032381,14.583333,35.047619,19.852381,14.583333,184,413,962,0,6.590799
3,Netclan20241019,Title: ROAS Dashboard for Campaign-Wise Google...,Client Background\nClient:\nA leading IT & tec...,client background client : leading & tech firm...,11.0,4.0,0.466667,0.044379,42.250000,38.165680,32.166272,42.250000,129,296,677,0,6.058632
